# HW5: Unsupervised Speech Recognition (USR)

Welcome to HW5 in Introduction to Deep Learning 11685. You will be working on Unsupervised Speech Recognition with GANs in this HW. You will be reimplementing and further improving on the model given in the USR paper by Facebook AI.<br>
Link: https://arxiv.org/abs/2105.11084


# Global Variables

In [ ]:
USE_DRIVE = False # using google drive or not, as GCP cannot connect to drive
SUBMIT = False # whether to submit to kaggle

# Installations

In [ ]:
! pip install git+https://github.com/pytorch/fairseq # need to install fairseq
# You can install other libraries such as torchsummaryX, wandb and so on

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pytorch/fairseq to /tmp/pip-req-build-u6yco_ml
  Running command git clone --filter=blob:none --quiet https://github.com/pytorch/fairseq /tmp/pip-req-build-u6yco_ml
  Resolved https://github.com/pytorch/fairseq to commit 3f6ba43f07a6e9e2acf957fc24e57251a7a3f55c
  Running command git submodule update --init --recursive -q
  From https://github.com/ngoyal2707/Megatron-LM
   * branch            adb23324c222aad0aad89308e70302d996a5eaeb -> FETCH_HEAD
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install torchsummaryX wandb

In [ ]:
if USE_DRIVE:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


# Kaggle

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle/

api_token = '{"username":"sharonxin1207","key":"xxx"}'

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write(api_token) # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [ ]:
!kaggle competitions download -c 11-685-s23-hw5
!mkdir '/content/data'

!unzip -qo '/content/11-685-s23-hw5.zip' -d '/content/data'
# move this folder to DeepLearningHW5GAN

# Imports

In [ ]:
import torch
from torch import nn, optim
from torch.utils import data
from torch.nn.utils.rnn import *

import numpy as np
from tqdm import tqdm
import sys
import json
import pprint

# add any other imports that you want 

has_cuda = torch.cuda.is_available()
if has_cuda:
  print("GPU: ", torch.cuda.get_device_name(0))
device = torch.device("cuda:0" if has_cuda else "cpu")
print("Device: ", device)

GPU:  Tesla T4
Device:  cuda:0


In [ ]:
# TODO
if USE_DRIVE:
  %cd /content/drive/MyDrive/DeepLearningHW5GAN
else:
  # this is the path for GCP save directory
  %cd /content/DeepLearningHW5GAN/


/content/DeepLearningHW5GAN


In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Dataset and DataLoaders

You have TODOs which need to be completed in `task/unpaired_audio_text.py` before you run these cells. You just need to replace the paths. You can use the original code base as a reference.



In [ ]:
from task import *
task = UnpairedAudioText()

/content/DeepLearningHW5GAN


In [ ]:
train_dataloader_args = dict(batch_size=160, #feel free to change these values
                             num_workers=4,
                            ) if has_cuda else dict(batch_size=64)
train_dataloader_args["shuffle"] = True
train_dataloader_args["collate_fn"] = task.datasets["train"].collater

validation_dataloader_args = train_dataloader_args.copy()
validation_dataloader_args["shuffle"] = False
validation_dataloader_args["collate_fn"] = task.datasets["valid"].collater

train_dataloader = data.DataLoader(task.datasets["train"], **train_dataloader_args)
validation_dataloader = data.DataLoader(task.datasets["valid"], **validation_dataloader_args)

In [ ]:
for x in train_dataloader:
  print(x['net_input']['features'].shape)
  break

torch.Size([160, 261, 512])


# Model and Training Configurations

You need to complete the TODOs in `model/wav2vec_u.py` before you run this cell. You can use the original codebase as a refernce to complete this.
Original Codebase: https://github.com/pytorch/fairseq/blob/main/examples/wav2vec/unsupervised/


In [ ]:
from model import Wav2vec_U

model = Wav2vec_U(task.target_dictionary).to(device)
print(model)

For a GAN, you need optimizers for both the discriminator and the generator. Configure the optimizers according to fairseq's configuration given in the link:
https://github.com/pytorch/fairseq/blob/main/examples/wav2vec/unsupervised/config/gan/w2vu.yaml


In [32]:
from itertools import chain

num_epochs = 1000
epoch_start = 1

if epoch_start == 1:
    # define 2 optimizers for different parts of the model at the start of the training
    optimizer = {
      "discriminator": optim.Adam(model.discriminator.parameters(),
                                  # TODO: define lr, weight decay, betas and other relavant parameters
                                  lr = 0.0005, betas=(0.5, 0.98), eps=1e-06, weight_decay=0.0001
                                 ),
      "generator": optim.Adam(chain(model.generator.parameters(), model.segmenter.parameters()),
                              # TODO: define lr, weight decay, betas and other relavant parameters
                               lr = 0.0004, betas=(0.5, 0.98), eps=1e-06, weight_decay=0.0
                              )
    }
    
# Optional TODO: Consider using mixed-precision to speed up training
scaler      = torch.cuda.amp.GradScaler()

A bunch of TODOs in the next cell. <br><br>
Tip: Instead of completing whole `run_model` function and the debugging while running the experiment section, you can create a new cell and code your own sanity check. It may help you to understand what is returned from the dataloader, what needs to be pushed to the device, how model is called and what `loss_stats` are.

In [33]:
# Hint: You may find pdb to be a great tool in helping you understand returned values
# from the dataloader and the model. Usage:
# import pdb
# pdb.set_trace()

def run_model(model, dataloader, optimizer):
    cumulative_stats = dict()
    # model.set_num_updates(1)

    for data in tqdm(dataloader, desc="Train" if model.training else "Eval "):
        net_input = data['net_input']
        #net_input = dict()
        net_input['features'] = net_input['features'].to(device)
        net_input['padding_mask'] = net_input['padding_mask'].to(device)
        
        data['net_input'] = net_input

        # What are the keys and values obtained from the data loader?
        # TODO: move all tensors to GPU
        # Tip: Checking what is inside net_input might help


        loss_stats = dict()

        if model.training:
            net_input['random_label'] = net_input['random_label'].to(device)
            # TODO: We are training the model. Might need to do something with the optimizer?
            # Remember that you are training a GAN. Both optimizers won't be used at the same time.
            # You may have to write an if statement or something similar to use the specific optimizer.
            # You may have to use the discrim_step() attribute in the Wav2vec_U class
            # curr_optimizer = optimizer[model.get_groups_for_update()]
            if model.discrim_step(model.update_num):
                optimizer['discriminator'].zero_grad()
            else:
                optimizer['generator'].zero_grad()
            # curr_optimizer.zero_grad()

            with torch.cuda.amp.autocast():
              loss_stats = model(**net_input) # forward pass
              
            total_loss = 0.0 
            for loss_type, loss_value in loss_stats['losses'].items():
              if loss_value is None:
                  continue
              else:
                  total_loss += loss_value
            total_loss /= net_input["features"].size(0) # average by batch

            group = model.get_groups_for_update() # told generator or de
            #print("Group Is")
            #print(group)
            # Look at what the get_groups_for_update() function does in the Wav2vec_U class
            # Can you try to think how you can use discrim_step() previously?

            # TODO: backprop loss and run the corresponding optimizer (Tip: See what 'group' is)
            scaler.scale(total_loss).backward()
            scaler.step(optimizer[group])
            scaler.update()
            #torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)

            # model.set_num_updates(model.update_num+1)
            torch.cuda.empty_cache()

        else:
            # validation
            loss_stats = task.valid_step(data, model)


        # accumulate batch stats
        for k, v in loss_stats.items():
            if type(v) is dict:
                # flatten inner dictionary
                key_value_pairs = [(k + "_" + kn, vn) for kn, vn in v.items()]
            else:
                key_value_pairs = [(k, v)]

          # TODO: accmulate all statistics into cumulative_stats, a dictionary
          # NOTE: you should convert any returned tensors to either values or numpy arrays
          # cumulative_stats shouldn't have nested dictionaries
            for i in range(len(key_value_pairs)):
              cum_key, cum_value = key_value_pairs[i]
              if cum_key not in cumulative_stats.keys():
                cumulative_stats[cum_key] = 0
              
              if cum_value is None:
                continue
              if torch.is_tensor(cum_value):
                cumulative_stats[cum_key] += cum_value.cpu().detach().numpy()
              else:
                cumulative_stats[cum_key] += cum_value
    
    # average stats over the dataset
    # Note that some metrics are already averaged over batch, so the result won't make sense
    # You can fix them if needed
    for k, v in cumulative_stats.items():
        if v is not None:
            v = v / len(dataloader.dataset)
        if type(v) is np.ndarray:
            v = v.tolist()
        cumulative_stats[k] = v

    return cumulative_stats

In [34]:
def save_model(model, optimizer, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'd_optimizer_state_dict'   : optimizer["discriminator"].state_dict(),
         "g_optimizer_state_dict"   : optimizer["generator"].state_dict(),
        #  'scheduler_state_dict'     : scheduler.state_dict(),
          'distance'                  : metric, 
         'epoch'                    : epoch}, 
         path
    )

def load_model(path, model, d_optimizer= None, g_optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if d_optimizer != None and g_optimizer != None:
        d_optimizer.load_state_dict(checkpoint['d_optimizer_state_dict'])
        g_optimizer.load_state_dict(checkpoint['g_optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        
    epoch   = checkpoint['epoch']
    metric  = checkpoint['distance']

    return [model, optimizer, scheduler, epoch, metric]

# Experiments

In [ ]:
# Login to Wandb
# Initialize your Wandb Run Here
# Save your model architecture in a txt file, and save the file to Wandb
import wandb
wandb.login(key="27ad915a9386068b1fc160cd97b84be7ba1fe659") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: Currently logged in as: wenxinz3 (sharonxin1207). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [35]:
run = wandb.init(
    name = "run-hw5", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "allow", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw5-ablations", ### Project should be created in your wandb account 
    # config = config ### Wandb Config for your run
)

edit_distance,▄▃▅▅▇▂▁▂▄▃▅▄▅▃▅▃▆▄▃▃▄█▄▅▁▄▅▁▃▇▅▅▃▃▄▆▄▅▃▅
edit_distance,42.409


In [ ]:
epoch_start = 1
num_epochs = 2000
best_dist = 100
eval_interval = 5 # evaluation after how many epochs?

In [22]:
RELOAD = False
reload_path = 'gan_1780.pt'
if RELOAD:
  model, optimizer, scheduler, epoch, best_dist = load_model(
      reload_path, model, optimizer["discriminator"], optimizer["generator"])
  epoch_start = epoch

In [36]:
print(f"Training for {num_epochs} epochs", file=sys.stderr)

for epoch in range(epoch_start, num_epochs + epoch_start):
    print(f"Epoch {epoch}", file=sys.stderr)
    model.train()
    # The model uses the epoch number to decide which part of the network to train
    model.set_num_updates(epoch) # Look at what this function does in the Wav2vec_U class

    train_stats = run_model(model, train_dataloader, optimizer)

    if epoch % eval_interval == eval_interval-1:

        model.eval()
        with torch.no_grad():
            eval_stats = run_model(model, validation_dataloader, optimizer)
            print("Saving model...")
            valid_dist = eval_stats['edit_distance']
            if valid_dist <= best_dist:
              save_model(model, optimizer, valid_dist, epoch, f'gan_{epoch+1}.pt')
              best_dist = valid_dist
            wandb.log({
                'edit_distance' : valid_dist,
             })
            print(eval_stats)
            
        # TODO: perhaps save your model and optimizer here
        # Tip: You can even save the model after every epoch along with the best model. 
        # This may help to continue training even if the best model is from a very early epoch.

    else:
        eval_stats = {}
      

    # TODO: Log training/eval statistics


Training for 1000 epochs
Epoch 1
Train: 100%|██████████| 82/82 [00:25<00:00,  3.24it/s]
Epoch 2
Train: 100%|██████████| 82/82 [00:20<00:00,  3.94it/s]
Epoch 3
Train: 100%|██████████| 82/82 [00:23<00:00,  3.47it/s]
Epoch 4
Eval : 100%|██████████| 7/7 [00:02<00:00,  3.25it/s]
Epoch 5


Saving model...
{'edit_distance': 113.971, '_num_chars': 125.089, '_num_pred_chars': 48.912, 'nsentences': 1.0, '_world_size': 0.007, '_lm_score_sum': 0.0, '_kaldi_score_sum': 0.0, '_word_lm_sum': 0.0, '_num_words': 0.0, '_vocab_seen': [0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.0, 0.007, 0.001, 0.007, 0.003, 0.003, 0.003, 0.0, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.0, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.005, 0.007, 0.007, 0.002, 0.007, 0.007, 0.003, 0.006, 0.001, 0.007, 0.007, 0.001, 0.006, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.0, 0.007, 0.002, 0.007, 0.007, 0.007, 0.007, 0.0, 0.007, 0.007, 0.007, 0.007, 0.0]}


Train: 100%|██████████| 82/82 [00:22<00:00,  3.63it/s]
Epoch 6
Train: 100%|██████████| 82/82 [00:20<00:00,  4.00it/s]
Epoch 7
Train: 100%|██████████| 82/82 [00:21<00:00,  3.76it/s]
Epoch 8
Train: 100%|██████████| 82/82 [00:19<00:00,  4.11it/s]
Epoch 9
Eval : 100%|██████████| 7/7 [00:02<00:00,  3.32it/s]
Epoch 10


Saving model...
{'edit_distance': 115.719, '_num_chars': 125.089, '_num_pred_chars': 34.486, 'nsentences': 1.0, '_world_size': 0.007, '_lm_score_sum': 0.0, '_kaldi_score_sum': 0.0, '_word_lm_sum': 0.0, '_num_words': 0.0, '_vocab_seen': [0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.0, 0.0, 0.0, 0.007, 0.003, 0.001, 0.007, 0.0, 0.007, 0.0, 0.007, 0.007, 0.007, 0.001, 0.0, 0.007, 0.005, 0.004, 0.007, 0.0, 0.007, 0.007, 0.004, 0.0, 0.0, 0.007, 0.002, 0.0, 0.0, 0.007, 0.007, 0.007, 0.007, 0.001, 0.007, 0.007, 0.007, 0.002, 0.0, 0.0, 0.006, 0.007, 0.007, 0.0, 0.001, 0.0, 0.0, 0.0]}


Train: 100%|██████████| 82/82 [00:19<00:00,  4.11it/s]
Epoch 11
Train: 100%|██████████| 82/82 [00:21<00:00,  3.82it/s]
Epoch 12
Train: 100%|██████████| 82/82 [00:19<00:00,  4.13it/s]
Epoch 13
Train: 100%|██████████| 82/82 [00:21<00:00,  3.85it/s]
Epoch 14
Eval : 100%|██████████| 7/7 [00:02<00:00,  3.23it/s]
Epoch 15


...

Train: 100%|██████████| 82/82 [00:20<00:00,  3.93it/s]
Epoch 1991
Train: 100%|██████████| 82/82 [00:24<00:00,  3.39it/s]
Epoch 1992
Train: 100%|██████████| 82/82 [00:20<00:00,  3.96it/s]
Epoch 1993
Train: 100%|██████████| 82/82 [00:23<00:00,  3.42it/s]
Epoch 1994
Eval : 100%|██████████| 7/7 [00:02<00:00,  3.07it/s]
Epoch 1995


Saving model...
{'edit_distance': 109.965, '_num_chars': 125.089, '_num_pred_chars': 109.909, 'nsentences': 1.0, '_world_size': 0.007, '_lm_score_sum': 0.0, '_kaldi_score_sum': 0.0, '_word_lm_sum': 0.0, '_num_words': 0.0, '_vocab_seen': [0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.0, 0.007, 0.007, 0.006, 0.007, 0.007, 0.007, 0.007, 0.0, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.0, 0.007, 0.007, 0.0, 0.007, 0.007, 0.0, 0.007, 0.007, 0.0, 0.007, 0.007, 0.007, 0.0, 0.0, 0.007, 0.007, 0.0, 0.007, 0.007, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}


Train: 100%|██████████| 82/82 [00:24<00:00,  3.39it/s]
Epoch 1996
Train: 100%|██████████| 82/82 [00:20<00:00,  3.98it/s]
Epoch 1997
Train: 100%|██████████| 82/82 [00:24<00:00,  3.38it/s]
Epoch 1998
Train: 100%|██████████| 82/82 [00:20<00:00,  3.97it/s]
Epoch 1999
Eval : 100%|██████████| 7/7 [00:02<00:00,  3.02it/s]
Epoch 2000


Saving model...
{'edit_distance': 111.644, '_num_chars': 125.089, '_num_pred_chars': 114.568, 'nsentences': 1.0, '_world_size': 0.007, '_lm_score_sum': 0.0, '_kaldi_score_sum': 0.0, '_word_lm_sum': 0.0, '_num_words': 0.0, '_vocab_seen': [0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.004, 0.0, 0.007, 0.0, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.0, 0.0, 0.0, 0.007, 0.007, 0.0, 0.007, 0.007, 0.0, 0.007, 0.007, 0.007, 0.007, 0.007, 0.007, 0.0, 0.0, 0.007, 0.007, 0.0, 0.007, 0.007, 0.0, 0.0, 0.007, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}


Train: 100%|██████████| 82/82 [00:20<00:00,  3.94it/s]


# Testing and submission to Kaggle

In [23]:
from dataset import extracted_features_dataset
test_path = '11-685-s23-hw5/test/'# TODO: PathToTest
testset = extracted_features_dataset.ExtractedFeaturesDataset(path=test_path,
                                                              split='test')

test_dataloader_args = train_dataloader_args.copy()
test_dataloader_args["shuffle"] = False
test_dataloader_args["collate_fn"] = testset.collater

test_dataloader = data.DataLoader(testset, **test_dataloader_args)

Write the `test_step` function which can be coded very similar to `valid_step` given in `task/unpaired_audio_text.py`

Write some code to evaluate and get the results. You are free to write the below cells however you want

In [24]:
model.eval()
results = []


for data in tqdm(test_dataloader, desc="Test"):
    # TODO
    data_input = data['net_input']
    net_input = dict()
    net_input['features'] = data_input['features'].to(device)
    net_input['padding_mask'] = data_input['padding_mask'].to(device)
    if 'random_label' in data_input.keys():
      net_input['random_label'] = data_input['random_label'].to(device)
    data['net_input'] = net_input

    logging, pred = task.test_step(data, model)
    results += pred

print(len(results))

Test:  14%|█▍        | 1/7 [00:01<00:06,  1.02s/it]

HYP: w ʌ n w ʌ z l æ f t s t ɪ f æ z ɐ ɹ ʌ ʃ ɪ n l ʌ n d ə n n̩ ɪ n ə ɪ n ɚ v iː n ɪ t aɪ m b ʌ k ɪ ŋ h æ m p ɜː h æ p s ɛ n t h ɪ t uː p æ ɹ ɪ s n n̩ æ z h iː d ð ə h ɔːɹ s ɪ z n h æ z n̩ ɛ n i w ʌ n b ɹ ɑː t ɐ l æ ɾ ɚ f ɔːɹ m iː n n̩ æ s t d aɪ t æ n j ə n iː ɡ ɚ l


Test: 100%|██████████| 7/7 [00:02<00:00,  3.33it/s]


In [26]:
# TODO: Replace the path and get the phoneme_map.json for mapping
with open("11-685-s23-hw5/test/phoneme_map.json", "r") as file:
    phon_map = json.load(file)

In [ ]:
predictions = []

for line in results:
    # TODO: Map results with phon_map
    phon_list = line.split()
    for j in range(len(phon_list)):
      phon_list[j] = phon_map[phon_list[j]]
    chars = "".join(phon_list)
    predictions.append(chars)

print(len(predictions))

1000


In [28]:
# TODO: Make the CSV and submit to kaggle
import pandas as pd

with open("random_submission.csv", 'w') as fh:
  fh.write('id,predictions\n') 
  for i in range(len(predictions)):
    fh.write(str(i)+ ',' + predictions[i] + "\n") 

df = pd.read_csv("random_submission.csv")
df.to_csv('submission.csv', index = False)

In [29]:
if SUBMIT:
  !kaggle competitions submit -c 11-685-s23-hw5 -f submission.csv -m "I made it!"

100% 118k/118k [00:00<00:00, 166kB/s]
Successfully submitted to Unsupervised Speech Recognition